<a href="https://colab.research.google.com/github/ThabisoEDSA/MBTI-Personality-Classifier/blob/master/EDSA_Thabz_Xente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


#### Xente Fraud Detection System Classification
>>> Author : Thabiso Mareletsa

#### Importing packages and Data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from scipy.special import boxcox1p
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [0]:
#### Let's read in our MBTI Headset with 'read_csv'
train  =  pd.read_csv('/content/gdrive/My Drive/Explore/Xente/training.csv' )
test = pd.read_csv('/content/gdrive/My Drive/Explore/Xente/testing.csv' )
sample = pd.read_csv('/content/gdrive/My Drive/Explore/Xente/sample_submission.csv' )

In [438]:
len(sample)

45019

In [0]:
xe_train = train.copy()
xe_test = test.copy()

In [440]:
len(xe_train)

95662

In [441]:
#### now let's check if there are any missing values in this dataset

xe_train.isnull().sum()

TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
dtype: int64

In [442]:
##The data has no missing values

### Let's count the number of fraudulent transanctions versus non fradulent trans

print('We have : ', train[train['FraudResult']==1]['FraudResult'].count(), 'fradulent transactions')

We have :  193 fradulent transactions


In [443]:
print('We have : ', train[train['FraudResult']==0]['FraudResult'].count(), 'non-fradulent transactions')

We have :  95469 non-fradulent transactions


In [444]:
train.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')

- as we can see we have a very imbalanced dataset as the number of non-fradulent transaction are way higher
- this implies we must aim to deal with thi issue

In [0]:
### what categories of services do we have

In [446]:
train['ProductCategory'].unique()

array(['airtime', 'financial_services', 'utility_bill', 'data_bundles',
       'tv', 'transport', 'ticket', 'movies', 'other'], dtype=object)

In [447]:
#### what service providers do we have

train['ProviderId'].unique()

array(['ProviderId_6', 'ProviderId_4', 'ProviderId_1', 'ProviderId_5',
       'ProviderId_3', 'ProviderId_2'], dtype=object)

In [448]:

#### what product
train['ProductId'].unique()

array(['ProductId_10', 'ProductId_6', 'ProductId_1', 'ProductId_21',
       'ProductId_3', 'ProductId_15', 'ProductId_11', 'ProductId_19',
       'ProductId_4', 'ProductId_5', 'ProductId_20', 'ProductId_9',
       'ProductId_24', 'ProductId_14', 'ProductId_2', 'ProductId_13',
       'ProductId_22', 'ProductId_8', 'ProductId_7', 'ProductId_27',
       'ProductId_12', 'ProductId_16', 'ProductId_23'], dtype=object)

In [449]:
#### There are 23 products on 
#### let's check how many channels do we have
train['ChannelId'].unique()

array(['ChannelId_3', 'ChannelId_2', 'ChannelId_1', 'ChannelId_5'],
      dtype=object)

In [450]:

train['TransactionStartTime'].unique()

array(['2018-11-15T02:18:49Z', '2018-11-15T02:19:08Z',
       '2018-11-15T02:44:21Z', ..., '2019-02-13T09:54:35Z',
       '2019-02-13T10:01:10Z', '2019-02-13T10:01:28Z'], dtype=object)

In [451]:
### what pricing stratergy is used 
train['PricingStrategy'].unique()

array([2, 4, 1, 0])

- Let's view our data as categorical and numerical column
- First let's chech the munerical columns

In [452]:
xe_train.select_dtypes(include=['int','int64','float']).columns
print(len(xe_train.select_dtypes(include=['int','int64','float']).columns))

5


In [453]:
xe_train.select_dtypes(include='object').columns
print(xe_train.select_dtypes(include='object').columns)

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'ProviderId', 'ProductId', 'ProductCategory',
       'ChannelId', 'TransactionStartTime'],
      dtype='object')


In [0]:
combined = pd.concat([xe_train,xe_test], sort=False)

In [0]:
#### dealing with the imbalance in the data


In [455]:
combined.isnull().sum()

TransactionId               0
BatchId                     0
AccountId                   0
SubscriptionId              0
CustomerId                  0
CurrencyCode                0
CountryCode                 0
ProviderId                  0
ProductId                   0
ProductCategory             0
ChannelId                   0
Amount                      0
Value                       0
TransactionStartTime        0
PricingStrategy             0
FraudResult             45019
dtype: int64

In [456]:
len(combined)

140681

In [457]:
### let's loo
values_count = combined.nunique().sort_values()
np.sum(values_count == 1)

2

In [458]:
#### now let's look at duplicates

duplicates = []
for i, ref in enumerate(combined.columns[:-1]):
    for other in combined.columns[i + 1:-1]:
        if other not in duplicates and np.all(combined[ref] == combined[other]):
            duplicates.append(other)    
len(duplicates)

0

In [0]:
### our train dataset had no missing values but the test dataset introduced some N/As
### therefore we have to fill them in with zeros 

combined['FraudResult']=combined['FraudResult'].fillna(0)

In [460]:
### Let's recheck our dataset for missing values

combined.isnull().sum()

TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
dtype: int64

In [461]:
#### Since we have no missing values lets lable encode ou categorical variables

###Let's label encode catecorical variables above

#Label Encoding


cols = ('TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'ProviderId', 'ProductId', 'ProductCategory',
       'ChannelId', 'TransactionStartTime')

# process columns, apply LabelEncoder to categorical features
for c in cols:
  if c in cols:
    
    lbl = LabelEncoder() 
    lbl.fit(list(combined[c].values)) 
    combined[c] = lbl.transform(list(combined[c].values))

# shape        
print('Shape combined: {}'.format(combined.shape))

Shape combined: (140681, 16)


In [418]:
combined.head(3)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,114984,68522,3286,4711,3786,0,256,5,1,0,2,1000.0,1000,0,2,0.0
1,111539,45765,4269,3144,3786,0,256,3,23,2,1,-20.0,20,1,2,0.0
2,58688,88319,3589,1357,4093,0,256,5,0,0,2,500.0,500,2,2,0.0


In [462]:
#### let's just deal with skewed data

num_feats = new_data.dtypes[new_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = new_data[num_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness = skewness.drop('FraudResult', 0)
skewness.head(15)


Skew in numerical features: 



,Skew
Amount,0.420457
Value,-0.024825
ProductCategory,-0.176640
ProductId,-1.609936
ProviderId,-1.848997
ChannelId,-2.027742
PricingStrategy,-3.378600
TransactionStartTime,-3.681659


In [463]:
#### let's just box cox transform this thing and see what happens

#Correct for skewness by using boxcox1p
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    new_data[feat] = boxcox1p(new_data[feat], lam)

There are 8 skewed numerical features to Box Cox transform


In [421]:
# I didn't use getdummies
#Get Dummies
#combined = pd.get_dummies(combined)
#len_train = train.shape[0]
#combined.head(3)
combined.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,114984,68522,3286,4711,3786,0,256,5,1,0,2,1000.0,1000,0,2,0.0
1,111539,45765,4269,3144,3786,0,256,3,23,2,1,-20.0,20,1,2,0.0
2,58688,88319,3589,1357,4093,0,256,5,0,0,2,500.0,500,2,2,0.0
3,71794,2629,4451,1318,7466,0,256,0,13,9,2,20000.0,21800,3,2,0.0
4,60900,71473,4269,3144,7466,0,256,3,23,2,1,-644.0,644,4,2,0.0


In [0]:
###
### As we can see boxcoxing this introduces nans in the value column
### we will replace that with mean of the column

combined['Amount']=combined['Amount'].fillna(combined['Amount'].mean())



In [465]:
#Based on data descriptions the following columns NA's will be filled with 'None'

combined.isnull().sum()

TransactionId           0
BatchId                 0
AccountId               0
SubscriptionId          0
CustomerId              0
CurrencyCode            0
CountryCode             0
ProviderId              0
ProductId               0
ProductCategory         0
ChannelId               0
Amount                  0
Value                   0
TransactionStartTime    0
PricingStrategy         0
FraudResult             0
dtype: int64

In [424]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140681 entries, 0 to 45018
Data columns (total 16 columns):
TransactionId           140681 non-null int64
BatchId                 140681 non-null int64
AccountId               140681 non-null int64
SubscriptionId          140681 non-null int64
CustomerId              140681 non-null int64
CurrencyCode            140681 non-null int64
CountryCode             140681 non-null int64
ProviderId              140681 non-null int64
ProductId               140681 non-null int64
ProductCategory         140681 non-null int64
ChannelId               140681 non-null int64
Amount                  140681 non-null float64
Value                   140681 non-null int64
TransactionStartTime    140681 non-null int64
PricingStrategy         140681 non-null int64
FraudResult             140681 non-null float64
dtypes: float64(2), int64(14)
memory usage: 18.2 MB


In [0]:
combined['FraudResult'] = combined.FraudResult.astype(int)

In [0]:
#### Drop a few columns let's see what happens 



In [467]:

combined.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,114984,68522,3286,4711,3786,0,256,5,1,0,2,1000.0,1000,0,2,0
1,111539,45765,4269,3144,3786,0,256,3,23,2,1,-20.0,20,1,2,0
2,58688,88319,3589,1357,4093,0,256,5,0,0,2,500.0,500,2,2,0
3,71794,2629,4451,1318,7466,0,256,0,13,9,2,20000.0,21800,3,2,0
4,60900,71473,4269,3144,7466,0,256,3,23,2,1,-644.0,644,4,2,0


In [0]:
len_train = len(xe_train)
X_train = combined[:len_train]
y_train = X_train.FraudResult.values
X_train = X_train.drop('FraudResult', 1)
X_test = combined[len_train:]
y_test = X_test.FraudResult.values
Xtest = X_test.drop(['FraudResult'], 1)


In [428]:
len(X_train)

95662

In [429]:
len(y_train)


95662

In [430]:
len(X_test)

45019

In [431]:
len(y_test)

45019

In [469]:
X_test.drop('FraudResult',1, inplace=True)
X_test.head(3)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,85796,67305,1603,3808,2064,0,256,4,20,0,2,1000.0,1000,94556,4
1,135249,78539,2711,1827,3194,0,256,4,6,2,2,2000.0,2000,94557,2
2,82191,111591,4269,3144,2064,0,256,3,23,2,1,-50.0,50,94558,2


In [470]:
### great now let's do the train test split


model = LogisticRegression()
#y = combined['FraudResult']
#X = combined.drop('FraudResult',1)


#X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size= 0.50,random_state=40)

#### train the model on the training data
model.fit(X_train,y_train)
y_hat = model.predict( X_test)
print('Accuracy Score:',accuracy_score(y_test, y_hat)*100)
#print(accuracy_score(y_hat,label_test1)*100)
print(log_loss(y_test,y_hat,labels=[0,1]))
#X_test.head(3)

Accuracy Score: 99.9022634887492
0.03375777657573566


In [434]:
len(y_hat)

45019

In [0]:
#Output to CSV
output_avg = pd.DataFrame({'A': xe_test.TransactionId, 'B': y_hat})
output_avg.to_csv('/content/gdrive/My Drive/Explore/Xente/submission1_xente.csv', index=False)